In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import random


In [2]:
if torch.cuda.is_available():
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor
    

In [3]:
data = open('texts/Lovecraft.txt','r').read()
alphabet = set(data)

ix_to_char = {k:v for k,v in enumerate(alphabet)}
char_to_ix = {k:v for v,k in enumerate(alphabet)}

In [4]:
NUM_LAYERS = 3
BATCH_SIZE = 1
HIDDEN_DIM = 128
SEQ_LEN = 65


In [5]:
def sequence_to_tensor(sequence):
    tensor = torch.zeros(SEQ_LEN,len(alphabet)).type(DTYPE)
    for i, c in enumerate(sequence):
        tensor[i][char_to_ix[c]] = 1
    return tensor


In [6]:
class LSTM(nn.Module):

    def __init__(self,alphabet_size, hidden_dim, output_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.i2h = nn.Linear(alphabet_size,hidden_dim)
        self.lstm = nn.LSTM(hidden_dim,hidden_dim,NUM_LAYERS)
        self.h2O = nn.Linear(hidden_dim, output_size)
        
        self.hidden = self.init_hidden()
        
        
    def init_hidden(self):
       
        return (autograd.Variable(torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_dim).type(DTYPE)),
                autograd.Variable(torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_dim).type(DTYPE)))

    def forward(self, sequence):
        out = self.i2h(sequence)
        lstm_out, self.hidden = self.lstm(out.view(SEQ_LEN-1,BATCH_SIZE,-1),self.hidden)
        out = self.h2O(lstm_out.view(-1,self.hidden_dim))
        return out
    
    
    def gen_text(self, sequence,t=None):
    
        inputs = autograd.Variable(sequence_to_tensor(sequence))
        idxs = torch.zeros(inputs.data.size())
        out = self(inputs[:-1,:])

        if t != None:
            soft_out = F.softmax(out/t,dim=1)
            
            for i in range(soft_out.size()[0]):
                idxs[i] = np.random.choice(soft_out.size()[1],p=soft_out.data.numpy()[i])
                
        else:
            idxs = out.max(1)[1].data

        
        return out,idxs
    


In [7]:
rnn = LSTM(len(alphabet),HIDDEN_DIM,len(alphabet)).type(DTYPE)
optimizer = optim.Adam(rnn.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()

epochs = 100

In [ ]:
def train(data):
    
    rnn.train(True)
    for epoch in range(epochs):

        losses = np.array([])
        
        for i in range(0,len(data),SEQ_LEN):

            rnn.zero_grad()
            rnn.hidden = rnn.init_hidden()
            
            inputs = autograd.Variable(sequence_to_tensor(data[i:i + SEQ_LEN]))
                        
            out = rnn(inputs[:-1,:])
                        
            _,target = inputs[1:,:].topk(1)
            
            loss = criterion(out.view(-1,len(alphabet)),target.view(-1))
            losses = np.append(losses,loss.data[0])
            
            loss.backward()
            optimizer.step()


        print("Epoch {}/{}\nLoss: {:.2f}".format(epoch+1,epochs,losses.mean()))
        print("="*15)


In [ ]:
train(data[:10000])

Epoch 1/100
Loss: 3.32
Epoch 2/100
Loss: 3.25
Epoch 3/100
Loss: 3.25
Epoch 4/100
Loss: 3.25
Epoch 5/100
Loss: 3.25
Epoch 6/100
Loss: 3.24
Epoch 7/100
Loss: 3.24


In [ ]:
string = ''  


rnn.train(False)

_ ,idxs = rnn.gen_text(data[0:0+SEQ_LEN])
    
#for i in range(100):
    
    #out = rnn(out)
    #soft_out = F.softmax(out/t,dim=1)

for c in idxs:
    string += ix_to_char[c]

print(string)           



#print(string,file=open('texts/output.txt','w'))

In [1]:
# overwrite dataloader
# pre-process txt file a bit
# hyperparameters
# refactor